<a href="https://colab.research.google.com/github/matteraggi/FineTuningAI/blob/main/vsProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --user transformers torch datasets peft tf-keras accelerate bitsandbytes trl

!pip install datasets --upgrade --force-reinstall

!pip install evaluate transformers openai human-eval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.3 MB/s eta 0:00:00
ERROR: Operation cancelled by user


In [1]:
# Importing stock ml libraries
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
# from transformers import BertTokenizer, BertModel, BertConfig
from datasets import load_dataset
from trl import SFTTrainer
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments,TrainingArguments,pipeline, logging,BitsAndBytesConfig
from peft import LoraConfig, PeftModel

In [3]:
# The model that you want to train from the Hugging Face hub
model_name = "bigcode/starcoder2-3b"  # Standard.  Good choice for a powerful and widely used LLM.

# The instruction dataset to use
dataset_name = "bigcode/self-oss-instruct-sc2-exec-filter-50k"  # Standard. A popular dataset for instruction tuning.

# Fine-tuned model name
new_model = "starcoder2-finetuned"  # Standard.  Clear and descriptive.

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension. It determines the size and parameter count of the low-rank adaptation
lora_r = 16  # Medium-high.  This is a reasonable value.  You could experiment with lower values (e.g., 32) for even more memory efficiency or higher values (e.g., 128) for potentially better performance (but more memory usage).

# Alpha parameter for LoRA scaling factor that determines the impact of the low-rank matrices on the original model's output.
# Controls the overall strength of the low-rank adaptation.
lora_alpha = 2  # Standard.  Often set to 2-4 times `lora_r`.  This is a common setting.

# Dropout probability for LoRA layers
lora_dropout = 0.05  # Standard.  A common dropout rate.  You could try slightly lower values (e.g., 0.05) if you're not seeing overfitting.

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True  # Standard. Essential for fitting large models on consumer GPUs.

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"  # Standard.  Using float16 can speed up computation if your GPU supports it.  If you encounter issues, try "bfloat16" (if your GPU supports it) or "float32".

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"  # Standard.  nf4 (NormalFloat4) is generally preferred.

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False  # Standard.  Double quantization can sometimes improve performance but increases complexity.  It's often left disabled initially.

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"  # Standard.

# Number of training epochs
num_train_epochs = 1  # Low.  One epoch is very low.  You'll almost certainly need more epochs (e.g., 3-5 or even more) to get good results.

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = True  # Standard.  You can enable this if your GPU supports it and `bnb_4bit_compute_dtype` is also set to a compatible value (float16).
bf16 = False  # Standard.  bf16 is generally preferred over fp16 if your GPU supports it.  Not available on a T4.

# Batch size per GPU for training
per_device_train_batch_size = 1  # Low.  You could try increasing this to 8 or even 16 if your GPU memory allows.  Higher batch sizes can often lead to faster training.

# Batch size per GPU for evaluation
per_device_eval_batch_size = 1  # Low.  Same as training batch size.

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 4  # Standard.  If you run out of memory, you can increase this to 2, 4, or higher.  This effectively increases the batch size.

# Enable gradient checkpointing
gradient_checkpointing = False  # CHANGED DA FRIGO DA PROBLEMI RISOLVERE Standard.  Essential for reducing memory usage, especially with larger models.

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3  # Standard.  A common value for gradient clipping.

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4  # Standard.  A good starting point.  You might want to experiment with values between 1e-4 and 5e-4.

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001  # Standard.

# Optimizer to use
optim = "paged_adamw_32bit"  # Standard.  A good optimizer.

# Learning rate schedule
lr_scheduler_type = "cosine"  # Standard.  A common and effective learning rate scheduler.

# Number of training steps (overrides num_train_epochs)
max_steps = 5000  # Standard.  Using `num_train_epochs` is usually sufficient.

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03  # Standard.

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True  # Standard.  Highly recommended for efficiency.

# Save checkpoint every X updates steps
save_steps = 500  # Low.  You should set this to a value greater than 0 (e.g., every 500 or 1000 steps) to save checkpoints during training.

# Log every X updates steps
logging_steps = 25  # Standard.

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = 1024  # Important.  You *must* set this.  A common value is 2048 or 4096, but it depends on your data and GPU memory.  Start with a lower value and increase it if possible.

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False  # Standard.  Can improve efficiency if your dataset has many short sequences.

# Load the entire model on the GPU 0
device_map = {"": 0}  # Standard.  This maps the model to the first GPU.

In [4]:
torch.cuda.empty_cache()

# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)


model.config.use_cache = False
model.config.pretraining_tp = 1
#if gradient_checkpointing:
#  model.gradient_checkpointing_enable()

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

# Load LoRA configuration
# Example configuration for target modules (can vary per model)
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"]  # These are common modules for transformers (queries and values in attention layers)
)


# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
    gradient_checkpointing=gradient_checkpointing,
)

# Inspect dataset columns
print(dataset.column_names)



README.md:   0%|          | 0.00/1.04k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/90.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/50661 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/12.1G [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

['fingerprint', 'sha1', 'seed', 'response', 'concepts', 'prompt', 'instruction', 'id']


In [5]:
def preprocess_function(examples):
    # Concatenate 'instruction' and 'prompt' for each example in the batch
    input_texts = []
    for instruction, prompt in zip(examples['instruction'], examples['prompt']):
        if instruction:  # If 'instruction' exists
            input_texts.append(instruction + " " + prompt)
        else:
            input_texts.append(prompt)

    # Tokenize the input texts
    return tokenizer(input_texts, padding='max_length', truncation=True, max_length=128)

# Apply tokenization
tokenized_dataset = dataset.map(preprocess_function, batched=True)



Map:   0%|          | 0/50661 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [ ]:
# Apply LoRA using PEFT
from peft import get_peft_model

for param in model.parameters():
    if param.dtype in [torch.float32, torch.float64, torch.float16, torch.bfloat16]:  # Check for floating point types
        param.requires_grad = True  # Ensure only floating-point parameters require gradients

print(any(param.requires_grad for param in model.parameters() if param.dtype in [torch.float32, torch.float64, torch.float16, torch.bfloat16]))  # Should print True

# Wrap the model with the LoRA configuration
model = get_peft_model(model, peft_config)


# Now, only LoRA layers will have requires_grad=True
print(any(param.requires_grad for param in model.parameters()))  # This should be True for LoRA layers

# Train the model with the modified configuration
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset,
    peft_config=peft_config,
    args=training_arguments,
)

# Start training
trainer.train()

# Save the fine-tuned model
trainer.model.save_pretrained(new_model)

True
True


Step,Training Loss
25,3.234500
50,3.448800
75,3.365400
100,3.023800
125,2.826600
150,2.303600
175,1.960500
200,1.841500
225,1.556800
250,1.519000


**NON RUNNARE DI DEFAULT**
Salvare modello che hai appena trainato sul tuo Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
model.save_pretrained("/content/drive/MyDrive/starcoder2-finetuned")
tokenizer.save_pretrained("/content/drive/MyDrive/starcoder2-finetuned")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


('/content/drive/MyDrive/starcoder2-finetuned/tokenizer_config.json',
 '/content/drive/MyDrive/starcoder2-finetuned/special_tokens_map.json',
 '/content/drive/MyDrive/starcoder2-finetuned/vocab.json',
 '/content/drive/MyDrive/starcoder2-finetuned/merges.txt',
 '/content/drive/MyDrive/starcoder2-finetuned/added_tokens.json',
 '/content/drive/MyDrive/starcoder2-finetuned/tokenizer.json')

**NON RUNNARE DI DEFAULT**
Per pushare il modello che hai appena trainato su git

In [ ]:
!apt-get install git
!git config --global user.email "mail"
!git config --global user.name "username"
yourPAT=""
!git clone https://github.com/matteraggi/FineTuningAI.git
#!cd FineTuningAI  # Go to the *existing* FineTuningAI directory
#!mv starcoder2-finetuned FineTuningAI/Model_Tensors/
!cd FineTuningAI && git add starcoder2_Model_Tensors
!cd FineTuningAI && git commit -m "Add fine-tuned model"
!git config --global credential.helper store  # Or 'cache' if you prefer
!cd FineTuningAI && git push https://{yourPAT}@github.com/matteraggi/FineTuningAI.git main


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.12).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.
On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean
Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 2 threads
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 16.10 MiB | 4.45 MiB/s, done.
Total 6 (delta 1), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/matteraggi/FineTuningAI.git
   84473d3..c0a8b3e  main -> main


**NON RUNNARE DI DEFAULT**
Loada il vecchio modello trainato da te dal tuo drive

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer #NOT TESTED

# Load the fine-tuned model from Google Drive
model_path = "/content/drive/MyDrive/starcoder2-finetuned"  # Path to your saved model

device = "cuda" if torch.cuda.is_available() else "cpu" # Get device

quantization_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_quant_type="nf4",  # Or "fp4" if you used that during training
      bnb_4bit_compute_dtype=torch.float16,  # Adjust if needed
      )
model = AutoModelForCausalLM.from_pretrained(
            model_path,
            device_map={"": device},
            quantization_config=quantization_config,
        ).eval()

# Load the model on GPU if available
model = model.to("cuda")


**NON RUNNARE DI DEFAULT**
Loada il vecchio modello da te dal git

In [14]:
import os # NOT TESTED
!git clone https://github.com/matteraggi/FineTuningAI.git
# Percorso alla directory del modello
model_dir = "FineTuningAI/starcoder2_Model_Tensors"

# Verifica se la directory esiste
if os.path.exists(model_dir):
    print("Directory del modello trovata!")

    from transformers import AutoModelForCausalLM, AutoTokenizer
    device = "cuda" if torch.cuda.is_available() else "cpu" # Get device

    quantization_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_quant_type="nf4",  # Or "fp4" if you used that during training
      bnb_4bit_compute_dtype=torch.float16,  # Adjust if needed
      )
    model = AutoModelForCausalLM.from_pretrained(
            model_dir,
            device_map={"": device},
            quantization_config=quantization_config,
        ).eval()

else:
    print(f"Errore: Directory del modello non trovata: {model_dir}")

fatal: destination path 'FineTuningAI' already exists and is not an empty directory.
Directory del modello trovata!


**Caricare il benchmark HumanEval**
Dopo il fine-tuning, devi confrontare il tuo modello con uno pre-addestrato (StarCoder2 senza fine-tuning) e con il modello fine-tunato.

In [16]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import os
import gc
import bitsandbytes as bnb

# Load HumanEval dataset
humaneval = load_dataset("openai/openai_humaneval", split="test")

quantization_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_quant_type="nf4",  # Or "fp4" if you used that during training
  bnb_4bit_compute_dtype=torch.float16,  # Adjust if needed
  )
device = "cuda" if torch.cuda.is_available() else "cpu" # Get device
# Load base model and tokenizer
base_model_name = "bigcode/starcoder2-3b"  # Base model
non_finetuned_model= AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map={"": device},
            quantization_config=quantization_config,
        ).eval()
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

model_path = "FineTuningAI/starcoder2_Model_Tensors"  # CAREFUL CAN CHANGE DEPENDING IF TESTING OLD OR NEW

def generate_code(current_model, prompt, is_finetuned=False):
    if not is_finetuned:
        system_prompt = "Generate Python code for the following task:\n"
        prompt = system_prompt + prompt  # Prepend system prompt for the base model

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = current_model.generate(**inputs,max_new_tokens=200)

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Evaluate base model and fine-tuned model
base_results = []
fine_tuned_results = []

for i in range(0, len(humaneval), 1): # Process 1 example at a time
    example = humaneval[i]
    prompt = example['prompt']

    base_code = generate_code(non_finetuned_model, prompt, is_finetuned=False)
    base_results.append({"prompt": prompt, "code": base_code})

    fine_tuned_code = generate_code(model, prompt, is_finetuned=True)
    fine_tuned_results.append({"prompt": prompt, "code": fine_tuned_code})

    gc.collect() # Force garbage collection

# Display some results
print("Base Model Results:", base_results[:3])
print("Fine-tuned Model Results:", fine_tuned_results[:3])


Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for o

KeyboardInterrupt: 

In [18]:
print("Base Model Results:", base_results[:3])
print("Fine-tuned Model Results:", fine_tuned_results[:3])

Base Model Results: [{'prompt': 'from typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n', 'code': 'Generate Python code for the following task:\nfrom typing import List\n\n\ndef has_close_elements(numbers: List[float], threshold: float) -> bool:\n    """ Check if in given list of numbers, are any two numbers closer to each other than\n    given threshold.\n    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)\n    False\n    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)\n    True\n    """\n\n# + [markdown] id="0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_0_

In [17]:
import json

# Assuming base_results and fine_tuned_results are your arrays
results = {
    "base_results": base_results,
    "fine_tuned_results": fine_tuned_results
}

# Save as JSON
with open("model_results.json", "w") as f:
    json.dump(results, f, indent=4)

# Save as text (optional)
with open("model_results.txt", "w") as f:
    f.write(str(results))


In [19]:
from google.colab import drive
drive.mount('/content/drive')
# Save as JSON in Drive
with open("/content/drive/MyDrive/model_results.json", "w") as f:
    json.dump(results, f, indent=4)

# Save as text in Drive
with open("/content/drive/MyDrive/model_results.txt", "w") as f:
    f.write(str(results))


Mounted at /content/drive


In [ ]:
!git add model_results.json
!git commit -m "Save model results"
!git push origin main  # Replace 'main' with your branch if necessary


In [ ]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "What is a large language model?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

NameError: name 'logging' is not defined